# Importing the required libraries

In [1]:
import pymongo
import csv
import json
import pandas as pd

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]

In [3]:
collection_names = ["Traffic_Crash", "speed_camera_violations"]

In [4]:
traffic_crashes_path = r"C:\Users\sruzoom\Dap Data sets\Traffic_Crashes_-_Crashes.json"

speed_camera_path = r"C:\Users\sruzoom\Dap Data sets\Speed_Camera_Violations.csv"

file_paths = [ traffic_crashes_path, speed_camera_path ]

In [9]:
with open(file_paths[0], 'r') as json_file:
    json_data = json.load(json_file)
    db[collection_names[0]].insert_many(json_data)

In [10]:
with open(file_paths[1], 'r') as csv_file:
    reader = csv.DictReader(csv_file)
    csv_data = list(reader)
    db[collection_names[1]].insert_many(csv_data)

# Checking the documents count in MongoDB

In [5]:
for collection_name in collection_names:
    count = db[collection_name].count_documents({})
    print(f"Total documents in {collection_name}: {count}")
    

Total documents in Traffic_Crash: 821716
Total documents in speed_camera_violations: 378235


# Checking the document Count in actual files

In [13]:
with open(traffic_crashes_path, 'r') as file:
    json_data = json.load(file)
total_records_json = len(json_data)
print(f"Total documents in Traffic_Crash: {total_records_json}")


df_speed_camera = pd.read_csv(speed_camera_path)
total_records_speed_camera = len(df_speed_camera)
print(f"Total documents in speed_camera_violations: {total_records_speed_camera}")

Total documents in Traffic_Crash: 821716
Total documents in speed_camera_violations: 378235


# Sorting with Date

In [16]:
# Dictionary to hold collection names and their respective date fields
collection_details = {
    "Traffic_Crash": "CRASH_DATE",
    "speed_camera_violations": "VIOLATION_DATE"
}

# Iterate over the dictionary to process each collection
for collection_name, date_field in collection_details.items():
    # Create an index on the date field for efficient sorting and querying
    db[collection_name].create_index([(date_field, pymongo.ASCENDING)])

    # Example to retrieve and display the first 5 documents sorted by date
    sorted_docs = db[collection_name].find().sort(date_field, pymongo.ASCENDING).limit(5)
    print(f"First 5 sorted documents in {collection_name}:")
    for doc in sorted_docs:
        print(doc)

First 5 sorted documents in Traffic_Crash:
{'_id': ObjectId('661d0f06145d4918c9956676'), 'CRASH_RECORD_ID': '1f810d065f794af2aa0a100a3e130fc4ce6443501a3900d72d80c08650351288ae835572600439502f1be7bbcf764294d1ae277e3310ea4930fb639494cb56cf', 'CRASH_DATE_EST_I': None, 'CRASH_DATE': '01/01/2016 01:00:00 AM', 'POSTED_SPEED_LIMIT': 30, 'TRAFFIC_CONTROL_DEVICE': 'UNKNOWN', 'DEVICE_CONDITION': 'UNKNOWN', 'WEATHER_CONDITION': 'UNKNOWN', 'LIGHTING_CONDITION': 'UNKNOWN', 'FIRST_CRASH_TYPE': 'PARKED MOTOR VEHICLE', 'TRAFFICWAY_TYPE': 'PARKING LOT', 'LANE_CNT': 1.0, 'ALIGNMENT': 'STRAIGHT AND LEVEL', 'ROADWAY_SURFACE_COND': 'UNKNOWN', 'ROAD_DEFECT': 'UNKNOWN', 'REPORT_TYPE': 'NOT ON SCENE (DESK REPORT)', 'CRASH_TYPE': 'NO INJURY / DRIVE AWAY', 'INTERSECTION_RELATED_I': None, 'NOT_RIGHT_OF_WAY_I': None, 'HIT_AND_RUN_I': None, 'DAMAGE': 'OVER $1,500', 'DATE_POLICE_NOTIFIED': '01/03/2016 08:00:00 PM', 'PRIM_CONTRIBUTORY_CAUSE': 'UNABLE TO DETERMINE', 'SEC_CONTRIBUTORY_CAUSE': 'NOT APPLICABLE', 'STREET